<h1> Custom Environment </h1>
<h4> -An attempt at creating it from scratch </h4>
<i> "You either die a hero, or live long enough to see yourself become a villain."</i>

class
<br>
initializer
<br>
step
<br>
reset
<br>
render
<br>
close
<br>
<br>
Validate thorugh check_env()

https://colab.research.google.com/github/araffin/rl-tutorial-jnrr19/blob/master/5_custom_gym_env.ipynb#scrollTo=9DOpP_B0-LXm

In [1]:
import numpy as np
import pandas as pd

import gymnasium as gym
from gymnasium import spaces

from stable_baselines3.common.env_checker import check_env

from stable_baselines3 import SAC
from stable_baselines3 import PPO

from stable_baselines3.common.evaluation import evaluate_policy

from RewardFunctions import (
    sharpe_ratio,
    sortino_ratio,
    calculate_drawdown,
    sterling_ratio,
    return_ratio,
    penalise_reward
)

In [2]:
history_usage = 30
rolling_reward_window = 10

return_data = pd.read_csv("../../Data/StockReturns.csv")

esg_data = np.array([36.6, 35.3, 17.9, 18, 
                    18, 21.2, 18.7, 29.2, 
                    15.7, 25.6, 25.6, 18.4, 
                    19.8, 13.8, 18.1, 19, 
                    17.2, 14, 17.2, 19.5, 
                    19.7, 21.2, 26.8, 19.3])

objective = "Sterling"

esg_compliancy = True

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

In [15]:
data = yf.download("AKRBP.OL", start="2020-12-01", end="2024-12-31")

[*********************100%***********************]  1 of 1 completed


In [19]:
# data["Volume"].tail()

In [18]:
# pd.DataFrame(np.repeat(data["Volume"].values,2))